In [22]:


%pip install webdriver-manager
%pip install lxml
%pip install selenium
%pip3 install webdriver_manager
%pip install --upgrade pip
%pip install -U selenium


  Obtaining dependency information for webdriver-manager from https://files.pythonhosted.org/packages/19/5a/a16653bfce685c9832217d377f52065351eeac9862e44e2996cd81f3bb4d/webdriver_manager-4.0.0-py2.py3-none-any.whl.metadata
  Using cached webdriver_manager-4.0.0-py2.py3-none-any.whl.metadata (11 kB)
  Using cached python_dotenv-1.0.0-py3-none-any.whl (19 kB)
Using cached webdriver_manager-4.0.0-py2.py3-none-any.whl (27 kB)
  Obtaining dependency information for lxml from https://files.pythonhosted.org/packages/31/58/e3b3dd6bb2ab7404f1f4992e2d0e6926ed40cef8ce1b3bbefd95877499e1/lxml-4.9.3-cp311-cp311-win_amd64.whl.metadata
  Using cached lxml-4.9.3-cp311-cp311-win_amd64.whl.metadata (3.9 kB)
Using cached lxml-4.9.3-cp311-cp311-win_amd64.whl (3.8 MB)
Note: you may need to restart the kernel to use updated packages.
  Obtaining dependency information for selenium from https://files.pythonhosted.org/packages/10/56/8288d1813a68c1e0638515dbb777fce6d87d0d240e683216f956145310e6/selenium-4.11.2-

UsageError: Line magic function `%pip3` not found.


In [23]:

from csv import writer

import time

from time import sleep

from random import randint

In [24]:
import selenium


In [25]:

from selenium import webdriver

from selenium.webdriver.chrome.service import Service as ChromeService

from webdriver_manager.chrome import ChromeDriverManager

In [26]:

from selenium.webdriver.common.by import By


from selenium.webdriver.support.ui import WebDriverWait



from selenium.common.exceptions import NoSuchElementException

In [27]:
option= webdriver.ChromeOptions()


option.add_argument("--incognito")



In [28]:

job_search_keyword =  ['Python+Developer']

location_search_keyword = ['India']

paginaton_url = 'https://in.indeed.com/jobs?q={}&l={}&filter=0&sort=date&start={}'

print(paginaton_url)

https://in.indeed.com/jobs?q={}&l={}&filter=0&sort=date&start={}


In [29]:
start = time.time()


job_='python+developer'
location='India'


driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()),options=option)


driver.get(paginaton_url.format(job_,location,0))


sleep(randint(2, 6))

p=driver.find_element(By.CLASS_NAME,'jobsearch-JobCountAndSortPane-jobCount').text
p = p.replace(',', '')

# Max number of pages for this search
max_iter_pgs=int(p.split(' ')[0])//1500 


driver.quit() # Closing the browser we opened


end = time.time()

print('Max Iterable Pages for this search:',max_iter_pgs)


Max Iterable Pages for this search: 7


In [30]:

start = time.time()


job_='python+developer'
location='India'


job_lst=[]
job_description_list_href=[]

job_description_list = []
salary_list=[]


driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()),
                         options=option)
sleep(randint(2, 6))


for i in range(0,max_iter_pgs):
    driver.get(paginaton_url.format(job_,location,i*10))
    
    
    sleep(randint(2, 4))

    job_page = driver.find_element(By.ID,"mosaic-jobResults")
    jobs = job_page.find_elements(By.CLASS_NAME,"job_seen_beacon") # return a list

    for jj in jobs:
        job_title = jj.find_element(By.CLASS_NAME,"jobTitle")


        job_lst.append([job_title.text,
        #job_title.find_element(By.CSS_SELECTOR,"a").get_attribute("href"),
        job_title.find_element(By.CSS_SELECTOR,"a").get_attribute("id"),      
        jj.find_element(By.CLASS_NAME,"companyName").text,       
        jj.find_element(By.CLASS_NAME,"companyLocation").text,
        jj.find_element(By.CLASS_NAME,"date").text,
        job_title.find_element(By.CSS_SELECTOR,"a").get_attribute("href")])
        

        try: # I removed the metadata attached to this class name to work!
            salary_list.append(jj.find_element(By.CLASS_NAME,"salary-snippet-container").text)

        except NoSuchElementException: 
            try: 
                salary_list.append(jj.find_element(By.CLASS_NAME,"estimated-salary").text)
                
            except NoSuchElementException:
                salary_list.append(None)
      
                


driver.quit() 


end = time.time()

print(end - start,'seconds to complete Query!')

# alternate way to grab the info for job description to make it faster:


64.2035026550293 seconds to complete Query!


In [35]:
%pip install pymongo

  Obtaining dependency information for pymongo from https://files.pythonhosted.org/packages/4d/88/7b9bfad1ad6269e0b9e0bb39d093aabc27837da4ccb93391cda68f580984/pymongo-4.5.0-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for dnspython<3.0.0,>=1.16.0 from https://files.pythonhosted.org/packages/f6/b4/0a9bee52c50f226a3cbfb54263d02bb421c7f2adc136520729c2c689c1e5/dnspython-2.4.2-py3-none-any.whl.metadata
  Using cached dnspython-2.4.2-py3-none-any.whl.metadata (4.9 kB)
   ---------------------------------------- 0.0/468.2 kB ? eta -:--:--
   -------------------------------- ------ 389.1/468.2 kB 12.2 MB/s eta 0:00:01
   ---------------------------------------- 468.2/468.2 kB 7.4 MB/s eta 0:00:00
Using cached dnspython-2.4.2-py3-none-any.whl (300 kB)
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import pymongo
from pymongo import MongoClient

# Replace these with your MongoDB connection details
mongo_uri = "mongodb://localhost:27017/"
database_name = "indeed"
collection_name = "jobs"

# Connect to MongoDB
client = MongoClient(mongo_uri)
db = client[database_name]
collection = db[collection_name]

# Insert job listings into the collection
for i in range(0,len(job_lst)):
    job_data = {
        "job_title": job_lst[i][0],
        "job_id": job_lst[i][1],
        "company_name": job_lst[i][2],
        "company_location": job_lst[i][3],
        "posting_date": job_lst[i][4],
        "job_description_url": job_lst[i][5],
        "salary":salary_list[i]
      
    }
    print(job_data)
    collection.insert_one(job_data)

# Close the MongoDB connection
client.close()

print("Job listings have been stored in MongoDB.")


In [31]:
job_lst[0:100]

[['Senior Design Engineer',
  'sj_cf002728426aaaf1',
  'ARM',
  'Bengaluru, Karnataka',
  'Posted\nJust posted',
  'https://in.indeed.com/pagead/clk?mo=r&ad=-6NYlbfkN0BgJnowPS_nFa6JvbNw1Ud-JjG_6nenis8YkFjCtkUlxoHXLw2_bm5yl4s91IBplAkYfVTfHzluHBPfbFyOej7EzaYPh4OXbXfZK1ROGTXsDkLIvm_ycuO6HTfg0-JlUFX73_j3TQ315TUvRfybBWol2fGyCbC-XRRxY8zzPBZg50o8aSGAJkR4sH0bM717v4kcn5BqXiTqIlu7DPg88olYEMJ-QIaYmGrYH7YBE3uPDtuoLWTKE5gKKLDwC05rvrR8oZ6IjgVvEni2v8PgAmHYM2hZkcAEzad4bWqdBmZkYuZ75VhNQeIlkTWgYWLeC4XV68WPlf5Yn8DhyogLc5WM5swlDPCImHa_EebbMRI8Wz7EFZosLTPmQUXPTNt-BLkRZlE5rV9Rg7yR5QbsT-0u165R0grOCG7s3Y5vA08AhgWBk8TbUP9ScvEN6RSiwezDRA_0wyI8TQ==&xkcb=SoB5-_M3LUJLkEgIrT0EbzkdCdPP&p=0&fvj=0&vjs=3'],
 ['Full Stack Developer',
  'job_1f9afff5ae46510c',
  'CoinDiary',
  'Coimbatore, Tamil Nadu',
  'Posted\nJust posted',
  'https://in.indeed.com/company/CoinDiary/jobs/Full-Stack-Developer-1f9afff5ae46510c?fccid=b492f9bf64389df9&vjs=3'],
 ['GTK Developer',
  'job_57d0cc8d122019a5',
  'Ethereal Machines',
  'Bengalur

In [33]:
salary_list[0:100]

[None,
 '₹5,00,000 - ₹6,00,000 a year',
 '₹40,000 - ₹80,000 a month',
 '₹35,000 - ₹80,000 a month',
 '₹8,086 - ₹15,000 a month',
 None,
 None,
 None,
 None,
 None,
 None,
 '₹3,40,708 - ₹16,04,852 a year',
 None,
 None,
 '₹50,000 - ₹70,000 a month',
 None,
 None,
 '',
 None,
 '',
 None,
 '',
 None,
 None,
 '',
 '',
 None,
 None,
 None,
 '',
 None,
 None,
 None,
 None,
 '₹10,000 - ₹30,000 a month',
 'From ₹3,00,000 a year',
 '₹30,000 - ₹50,000 a month',
 '₹8,00,000 - ₹11,00,000 a year',
 None,
 '₹6,00,000 - ₹12,00,000 a year',
 None,
 None,
 '₹20,000 - ₹40,000 a month',
 None,
 '₹4,00,000 - ₹4,50,000 a year',
 None,
 None,
 None,
 None,
 None,
 '₹20,000 - ₹30,000 a month',
 None,
 '₹12,00,000 - ₹13,50,000 a year',
 'From ₹8,000 a month',
 None,
 '₹3,00,000 - ₹4,20,000 a year',
 None,
 None,
 None,
 None,
 None,
 '₹35,000 - ₹65,000 a month',
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 '₹20,00,000 - ₹25,00,000 a year',
 None,
 '₹20,00,000 - ₹25,00,000 a ye

In [32]:
%pip install numpy

In [1]:
import re
import numpy as np

# Sample job data
job_data = [
{'job_title': 'Python Developer', 'job_id': 'job_326ca37696299ae4', 'company_name': 'RGC', 'company_location': 'Raipur, Madhya Pradesh', 'posting_date': 'Posted\nJust posted', 'job_description_url': 'https://in.indeed.com/company/RGC/jobs/Python-Developer-326ca37696299ae4?fccid=959fd4799e3f8cd0&vjs=3', 'salary': '₹15,000 - ₹45,000 a month'},
{'job_title': 'Software Developer', 'job_id': 'job_24ebb212bd508c42', 'company_name': 'SUPRA CONTROLS PRIVATE LIMITED', 'company_location': 'Chennai, Tamil Nadu', 'posting_date': 'Posted\nJust posted', 'job_description_url': 'https://in.indeed.com/company/Supra-Controls-Private-Limited/jobs/Software-Engineer-24ebb212bd508c42?fccid=77650624d73f5326&vjs=3', 'salary': '₹3,60,000 - ₹4,50,000 a year'},
{'job_title': 'Amazon SP-API Developer', 'job_id': 'job_7176c36c21554184', 'company_name': 'Pappytech Solutions Pvt Ltd', 'company_location': 'Remote in Mumbai, Maharashtra', 'posting_date': 'Posted\nJust posted', 'job_description_url': 'https://in.indeed.com/company/Pappytect-Solutions-Pvt-Ltd/jobs/Developer-7176c36c21554184?fccid=87365259671e5d05&vjs=3', 'salary': '₹15,000 - ₹75,000 a month'},
{'job_title': 'Software Developer', 'job_id': 'job_5347647f565982ac', 'company_name': 'ESGATE', 'company_location': 'Remote in Bengaluru, Karnataka', 'posting_date': 'Posted\nJust posted', 'job_description_url': 'https://in.indeed.com/company/ESGATE/jobs/Software-Engineer-5347647f565982ac?fccid=5cbceee7e6aaaa9e&vjs=3', 'salary': 'From ₹10,79,224 a year'},
{'job_title': 'ERPNext Developer', 'job_id': 'job_4cf897af4ef9c8aa', 'company_name': 'Simpel Techlabs Pvt. Ltd.', 'company_location': 'Bengaluru, Karnataka', 'posting_date': 'Posted\nJust posted', 'job_description_url': 'https://in.indeed.com/company/Simpel-Techlabs-Pvt.-Ltd./jobs/Developer-4cf897af4ef9c8aa?fccid=fab59a62d4c3cdf6&vjs=3', 'salary': '₹20,000 - ₹30,000 a month'},
{'job_title': 'Python/Django Developer', 'job_id': 'job_486fdac0a261bb19', 'company_name': 'Raneso Global Pvt. Ltd.', 'company_location': 'Ranchi, Jharkhand', 'posting_date': 'Posted\nJust posted', 'job_description_url': 'https://in.indeed.com/company/Raneso-global-Pvt.-Ltd./jobs/Python-Developer-486fdac0a261bb19?fccid=f6a4e676365b2e6a&vjs=3', 'salary': '₹2,40,000 - ₹3,60,000 a year'},
{'job_title': 'Python Programmer', 'job_id': 'job_00eabdb7954caab1', 'company_name': 'AMPYR Energy Tech Solutions Pvt. Ltd.', 'company_location': 'New Delhi, Delhi', 'posting_date': 'Posted\nJust posted', 'job_description_url': 'https://in.indeed.com/company/AMPYR-Energy-Tech-Solutions-Pvt.-Ltd./jobs/Python-Developer-00eabdb7954caab1?fccid=3a6aebb79fcdbb06&vjs=3', 'salary': 'From ₹35,000 a month'},
{'job_title': 'Python Developer', 'job_id': 'job_c9e1d4fc4ab94605', 'company_name': 'iMaker technology Pvt. Ltd', 'company_location': 'Gandhinagar, Gujarat', 'posting_date': 'Posted\nJust posted', 'job_description_url': 'https://in.indeed.com/company/iMaker-technology-Pvt.-Ltd/jobs/Python-Developer-c9e1d4fc4ab94605?fccid=38779e8bc6512239&vjs=3', 'salary': '₹20,000 - ₹30,000 a month'},
{'job_title': 'Web Developer', 'job_id': 'job_c2c16bd7d9561c32', 'company_name': 'Pixselo Technologies Private Limited', 'company_location': 'Bengaluru, Karnataka', 'posting_date': 'Posted\nJust posted', 'job_description_url': 'https://in.indeed.com/company/Pixsello/jobs/Web-Developer-c2c16bd7d9561c32?fccid=6bce30b6d51ee145&vjs=3', 'salary': '₹2,40,000 - ₹3,00,000 a year'},
{'job_title': 'Python Developer', 'job_id': 'job_bea0578363e6b96c', 'company_name': 'BIG IT JOBS', 'company_location': 'Mumbai, Maharashtra', 'posting_date': 'Posted\nJust posted', 'job_description_url': 'https://in.indeed.com/rc/clk?jk=bea0578363e6b96c&fccid=bf03f340679e2b9b&vjs=3', 'salary': '₹50,000 - ₹1,08,000 a month'},
{'job_title': 'Jr Software developer', 'job_id': 'job_b50ef7bdd375675b', 'company_name': 'BIG IT JOBS', 'company_location': 'Bengaluru, Karnataka', 'posting_date': 'Posted\nJust posted', 'job_description_url': 'https://in.indeed.com/rc/clk?jk=b50ef7bdd375675b&fccid=bf03f340679e2b9b&vjs=3', 'salary': None},
{'job_title': 'Web Developer (advanced SQL, C# and/or Python experience)', 'job_id': 'job_2e6bee4661756eab', 'company_name': 'Zensar Technologies', 'company_location': 'Pune, Maharashtra', 'posting_date': 'Posted\nJust posted', 'job_description_url': 'https://in.indeed.com/rc/clk?jk=2e6bee4661756eab&fccid=e223a70f4b467dcd&vjs=3', 'salary': None},
{'job_title': 'Full Stack Developer', 'job_id': 'job_bf68bee05d11f4fa', 'company_name': 'BDIPlus', 'company_location': 'Bengaluru, Karnataka', 'posting_date': 'Posted\nJust posted', 'job_description_url': 'https://in.indeed.com/company/BDIPlus/jobs/Full-Stack-Developer-bf68bee05d11f4fa?fccid=ef6cc92f6d5fec02&vjs=3', 'salary': 'From ₹4,29,054 a year'},
{'job_title': 'Software Engineer II', 'job_id': 'job_d523a01b54b9ad39', 'company_name': 'Electronic Arts', 'company_location': 'Hyderabad, Telangana', 'posting_date': 'Posted\nJust posted', 'job_description_url': 'https://in.indeed.com/rc/clk?jk=d523a01b54b9ad39&fccid=617d7f961cfcf54a&vjs=3', 'salary': None},
{'job_title': 'Member of Technical Staff - 3', 'job_id': 'job_564b0779cd4719fa', 'company_name': 'Nutanix', 'company_location': 'Bengaluru, Karnataka', 'posting_date': 'Posted\nJust posted', 'job_description_url': 'https://in.indeed.com/rc/clk?jk=564b0779cd4719fa&fccid=dd7de5cf522ca5fc&vjs=3', 'salary': None},
{'job_title': 'Python Developer', 'job_id': 'job_38bfef4146be556a', 'company_name': 'Luminous Infoways Pvt. Ltd.', 'company_location': 'Bhubaneshwar, Orissa', 'posting_date': 'Posted\nJust posted', 'job_description_url': 'https://in.indeed.com/rc/clk?jk=38bfef4146be556a&fccid=66bd6b1635c0f9e9&vjs=3', 'salary': None},
{'job_title': 'Web Developer (Full Stack Developer)', 'job_id': 'job_eb385204a33dc9ae', 'company_name': 'APAC Business Headlines', 'company_location': 'Bengaluru, Karnataka', 'posting_date': 'Posted\nJust posted', 'job_description_url': 'https://in.indeed.com/company/APAC-BUSINESS-HEADLINES/jobs/Full-Stack-Developer-eb385204a33dc9ae?fccid=b0f318d5956409f9&vjs=3', 'salary': '₹4,50,000 - ₹7,20,000 a year'},
{'job_title': 'Python- Frappe Developer', 'job_id': 'job_432b99e14c24a774', 'company_name': 'Induct', 'company_location': 'Indore, Madhya Pradesh', 'posting_date': 'Posted\nJust posted', 'job_description_url': 'https://in.indeed.com/company/Induct/jobs/Python-Developer-432b99e14c24a774?fccid=8c8ceae97a5a1871&vjs=3', 'salary': '₹25,000 - ₹50,000 a month'},
{'job_title': 'Senior Member Technical Staff, Database Globalization', 'job_id': 'job_2851f2dd18872b72', 'company_name': 'Oracle', 'company_location': 'Bengaluru, Karnataka', 'posting_date': 'Posted\nJust posted', 'job_description_url': 'https://in.indeed.com/rc/clk?jk=2851f2dd18872b72&fccid=cd22d01053af7669&vjs=3', 'salary': None},
{'job_title': 'Backend Developer Intern', 'job_id': 'job_835baa8a27e7a65d', 'company_name': 'BIG IT JOBS', 'company_location': 'India', 'posting_date': 'Posted\nJust posted', 'job_description_url': 'https://in.indeed.com/rc/clk?jk=835baa8a27e7a65d&fccid=bf03f340679e2b9b&vjs=3', 'salary': None},
{'job_title': 'Full Stack Wordpress Developer', 'job_id': 'job_cbe536283382349a', 'company_name': 'Shanubogh Enterprises Private limited', 'company_location': 'Bengaluru, Karnataka', 'posting_date': 'Posted\nJust posted', 'job_description_url': 'https://in.indeed.com/company/Shanubogh-Enterprises-Private-Limited/jobs/Full-Stack-Developer-cbe536283382349a?fccid=4ac5c9c18ab50a04&vjs=3', 'salary': '₹3,00,000 - ₹4,00,000 a year'},
{'job_title': 'Full Stack wordpress Developer', 'job_id': 'job_f1ba2d6187ac89d5', 'company_name': 'Shanubogh Enterprises Private limited', 'company_location': 'Bengaluru, Karnataka', 'posting_date': 'Posted\nJust posted', 'job_description_url': 'https://in.indeed.com/company/Shanubogh-Enterprises-Private-Limited/jobs/Full-Stack-Developer-f1ba2d6187ac89d5?fccid=4ac5c9c18ab50a04&vjs=3', 'salary': 'From ₹2,40,000 a year'},
{'job_title': 'Python/Django Developer', 'job_id': 'job_6b1feb398bf6285a', 'company_name': 'ClickNPay Digital Solution India Pvt Ltd', 'company_location': 'Noida, Uttar Pradesh', 'posting_date': 'Posted\nJust posted', 'job_description_url': 'https://in.indeed.com/company/ClickNPay-Digital-Solution-India-Pvt-Ltd/jobs/Python-Developer-6b1feb398bf6285a?fccid=65e6e59d6e45f0ec&vjs=3', 'salary': '₹20,000 - ₹40,000 a month'},
{'job_title': 'Quality Assurance Analyst 2', 'job_id': 'job_ef0496d9d8adafb9', 'company_name': 'Epsilon', 'company_location': 'Bengaluru, Karnataka', 'posting_date': 'Posted\nJust posted', 'job_description_url': 'https://in.indeed.com/rc/clk?jk=ef0496d9d8adafb9&fccid=beb2b679d6e2402e&vjs=3', 'salary': None},
{'job_title': 'Informatica admin and support', 'job_id': 'job_cddcd956ade8cbfe', 'company_name': 'Unisys', 'company_location': 'Bengaluru, Karnataka', 'posting_date': 'Posted\nJust posted', 'job_description_url': 'https://in.indeed.com/rc/clk?jk=cddcd956ade8cbfe&fccid=189b4377c880f7e5&vjs=3', 'salary': None}
]
bengaluru_jobs = [job for job in job_data if "Bengaluru" in job["company_location"]]


salaries = []
for job in bengaluru_jobs:
    salary_text = job.get("salary")
    if salary_text:
        match_month = re.search(r"₹([\d,]+) - ₹([\d,]+) a month", salary_text)
        match_year = re.search(r"From ₹([\d,]+) a year", salary_text)
        
        if match_month:
            lower_salary = int(match_month.group(1).replace(",", ""))
            upper_salary = int(match_month.group(2).replace(",", ""))
            avg_salary_per_month = (lower_salary + upper_salary) / 2 
            salaries.append(avg_salary_per_month)
        elif match_year:
            salary_per_year = int(match_year.group(1).replace(",", ""))
            avg_salary_per_month = salary_per_year / 12
            salaries.append(avg_salary_per_month)

# Calculate average salary using numpy
average_salary = np.mean(salaries) if salaries else 0

print(f"Average Salary per Month for Bengaluru Jobs: ₹{average_salary:.2f} per month")


Average Salary per Month for Bengaluru Jobs: ₹42672.46 per month
